In [ ]:
import numpy as np
from numba import jit

import matplotlib.pyplot as plt

from functools import partial

import sys
sys.path.append('../miniMD')
from miniMD import *

# Temperature Replica Exchange Molecular Dynamics

In TREMD, we simulate the same system at various temperatures and attempt to exchange replicas every once in a while. The idea here is that in higher temperatures we have shorter correlation times since the system can cross energetic barriers significantly faster. By exchanging these decorrelated configurations with lower temperature replicas, the efficiency of the simulation is increased.

However, we need to make sure that the exchanges are made in such a way that the Boltzmann distribution in each replica is preserved:

$$p_i(x) = Z_i^{-1} e^{-\beta_i U(x)}$$

where $i$ denotes the index of the replica and $\beta_i \equiv 1/k_B T_i$ is the temperature corresponding the the $i$-th replica.

This is made sure by exchaning according to a Metropolis acceptance criterion given by:

$$p_{\text{acc}}(i \leftrightarrow j) = \min\left\{ 1, \exp[- \Delta U_{ij} (\beta_i - \beta_j)] \right\}$$

where $i \leftrightarrow j$ denotes a swap between replicas $i$ and $j$. For calculating this probability, we need to know the difference between the betas and the potential energy difference between the configurations of each replica:

$$\Delta U_{ij} = U(x_i) - U(x_j)$$




We start with a provided force and energy of a 2D system:

In [ ]:

@jit(nopython=True) 
def custom_potential_energy(current_x : np.ndarray) -> float:
    """
    Calculates the potential energy given a configuration current_x. 
    
    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    U : float
        Potential energy of the configuration

    """
    return 10 * ((current_x[0]**2 - 1)**2 + (current_x[0] - current_x[1])**2)


@jit(nopython=True) 
def custom_force_function(current_x : np.ndarray) -> np.ndarray:
    """
    Calculates the force given a configuration current_x. 

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    force : np.ndarray
        Force corresponding the provided configuration.

    """
    force = np.zeros(2)
    
    force[0] = -20 * (2 * current_x[0]**3 -current_x[0] - current_x[1])
    force[1] = 20 * (current_x[0] - current_x[1])

    return force


Next up here are some parameters for the simulation, notice especially the REMP specific ones. The list beta_range describes the betas for the different replicas. Remember, small beta = large temperature and vice versa. The exchange frequency should describe how often an exchange is attempted. N_replicas just stores the overall number of replicas.

In [ ]:
total_steps = 50000
equilibration_steps = 1000
output_frequency = 1

# REMD specific
beta_range = [0.1, 0.2, 0.4, 0.8, 1]
exchange_frequency = 100
N_replicas = len(beta_range)

beta = 1
timestep = 0.001
diffusion_coefficient = 1

initial_x = np.zeros(2)

assert equilibration_steps < total_steps, "Make sure you don't equilibrate longer than you simulate."
assert output_frequency < total_steps, "Make sure you don't output less often than you simulate."
assert exchange_frequency < total_steps, "Make sure you don't exchange less often than you simulate."
assert output_frequency > 0, "The output frequency needs to be larger than 0"


### Simulating replicas without exchanges

Now comes the implementation part. From the last tutorials you should have an idea how to code a small MD. As a first step towards REMD, we can simulate the different replicas without exchanging between them.

**1) Write a code that simulates the replicas without exchanges between them.** TIP: For each step, propagate each replica one timestep. NOT the other way around, which would be to simulate for N steps each replica.

In [ ]:
# Intialize output variables

...

# Run simulations

for step in range(total_steps):

    for rep in range(N_replicas):
        
        ...

**2) Plot the trajectories of each replicas side-by-side or on top of each other. Describe what differentiates each of them.**

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4), dpi=180)


for i in range(N_replicas):

    ...


x_values = np.linspace(-2.6, 2.6, 100)
y_values = np.linspace(-2.6, 2.6, 100)

x_grid, y_grid = np.meshgrid(x_values, y_values)
energies = np.zeros((len(x_values), len(y_values)))

for i in range(len(x_values)):
    for j in range(len(y_values)):

        energies[i,j] = custom_potential_energy(np.array([x_grid[i, j], y_grid[i, j]]))

ax.contourf(x_grid, y_grid, energies, levels=np.linspace(0, 20, 10), cmap="RdBu_r")

ax.legend(frameon=False, loc="upper left")
ax.set_xlabel("x")
ax.set_ylabel("y")

plt.show()

**3) Calculate the free energy along x using Boltzmann inversion. Compare the free energies and describe the behaviour at each temperature.**

In [ ]:
replica_free_energies = []
bins = np.linspace(-2,2,100)
bin_centers = (bins[1:] + bins[:-1]) / 2

for i in range(N_replicas):

    ...

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

for i, d in enumerate(replica_free_energies):

    ax.plot(bin_centers, d - np.min(d))
    
ax.set_xlabel("x")
ax.set_ylabel(r"F$_{bias}$(x)")
plt.show()

### Replica Exchange

Now we are ready to add exchanges to our simulation code. We here want the simplest code possible, therefore I'd recommend to:

1) Every exchange_frequency steps:
2) Draw a random integer i and j both within [0, N_replicas[
3) Calculate the acceptance criterion mentioned in the introduction
4) If accepted, swap the configurations of the two simulations. 

Regarding 4), swapping beta is sometimes also done in simulation codes. While both are physically correct, it depends on whether you want a continuous trajectory or a trajectory at constant temperature for each replica.

**4) Implement a replica exchange MD code. Try to reuse the code above and see how you can accomodate the exchange code.**


In [ ]:
# This array should track all exchanges. At the end oth the simulation, 
# exchange_matrix[i,j] should count how often you have selected i and switched it with j.

exchange_matrix = np.zeros((N_replicas, N_replicas), dtype=int)

In [ ]:
# Initialize Output Variables

...

# Run REMD

for step in range(total_steps):

    for rep in range(N_replicas):
        
        ...

    # Skip the exchange code below if step is not divisible by exchange_frequency
    if step % exchange_frequency != 0: continue

    #
    #   Attempt Exchange
    #

    ...

**5) Plot the trajectories of each replicas side-by-side or on top of each other. Describe the difference to the above plot without exchanges.**

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4), dpi=180)

ax.contourf(x_grid, y_grid, energies, levels=np.linspace(0, 20, 10), cmap="RdBu_r")

for i in range(N_replicas):

    ...

ax.legend(frameon=False, loc="upper left")
ax.set_xlabel("x")
ax.set_ylabel("y")

plt.show()

**6) Calculate the free energy along x using simple Boltzmann inversion. Compare the free energies to the ones without exchanges.**

In [ ]:
replica_free_energies = []
bins = np.linspace(-2,2,100)
bin_centers = (bins[1:] + bins[:-1]) / 2

for i in range(N_replicas):

    ...

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

for i, d in enumerate(replica_free_energies):

    ax.plot(bin_centers, d - np.min(d))
    
ax.set_xlabel("x")
ax.set_ylabel(r"F$_{bias}$(x)")
plt.show()

**7) Visualize the exchange matrix with a proper scale and axis labels.**

**8) Describe which replicas exchanged most and least and give an explanation for this outcome.**

In [ ]:
...

### Optional

- Besides TREMD, there is the massively successful Hamiltonian replica exchange. Here, you can exchange between two systems of different potential energy functions. This is VERY nice in case you have e.g. a biased system and an unbiased one, as in metadynamics or umbrella sampling. Also in free energy perturbation, one can use this trick to decrease correlations between samples. The exchange criterion is as easy as the TREMD one, see equation (5) here, where you can even cancel the kT's:
https://manual.gromacs.org/documentation/2021/reference-manual/algorithms/replica-exchange.html
Try to implement such an exchange scheme, you can recycle code from the FEP tutorial.
